In [1]:
%matplotlib inline

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

sns.set(style="whitegrid")

# Description to Modeling

In the previous chapter we calculated descriptive statistics for each variable.
For numerical variables, this usually included the (arithmetic) mean; often just called "the average".
We described the mean as the center of gravity of the data.
We also talked about the dispersion of the data in terms of the variance (or standard deviation).
It's possible for two data sets to have the same mean but very different variability.
Still, our main goal during EDA was simply to describe the data.

In this chapter, we want to make a subtle shift from merely describing the data we have to making conjectures about unseen data.
The conventional approach describes the process this way:

1. We take a random sample of data from a population.
2. We calculate the mean of the sample. If this is as far as we go, this is simply descriptive statistics.
3. We can use this mean to make inferences about the population mean.

So in Step 3 we move from descriptive statistics to inferential statistics.
There's a lot of baggage tied up with this approach.
For example, there is a focus on *error*.
If our estimate is off, it is because of sampling *error* or measurement *error*.
This terminology originates with astronomy when the devices of the day had poor resolution and multiple measurements were often averaged.
This is not to say there isn't measurement error today but that variation in our estimates arise from more than just "error".

Second, we don't always engage in random sampling.
We'll discuss some experimental design in a later chapter but the King County, Washington Home Sales data is not a random sample.
The data set is for the home sales for May 2014 to May 2015. 
All of them.
Rideshare data available on Kaggle?
The data set is for all the bicycles rented in a specific city during a specific period of time.
And this is true of a great deal of the data that data scientists work with.
Data scientists often work with "all" the data.

However, it's not really *all* of the data there ever will be.
It's a subset of historical data and in this sense, it's still a sampling, it's just not random.
This may have important ramifications for the conclusions we can draw from our data: the kinds of models we can build and what they might apply to.
Because Frequentist statistics depends on *random* samples, it must often go through mental gyrations to make these problems fit that experimental mold.
For Bayesian statistics, we just acknowledge we have observations for a process and we want to make inferences about unseen values of that process.
Whether they are unseen because we really did collect only a subset of all the current data or because more data will come in the future, it doesn't matter.
In a word, everything in Bayesian statistics is a model.

## What's a Model Anyway?

> "A model which took account of all the variegation of reality would be of no more use than a map at the scale of one to one." -- Joan Robinson

> "All models are wrong but some are useful" -- George Box

> "The map is not the territory" -- Alfred Korzybski

As the first quote suggests, a model that took all the variables into account would be as unwieldy as the process we're trying to model. Think of a 1:1 map of the United States. How would you even use it? Where would you store it? The next quote by George Box suggests that we need to be pragmatic about our model building. All of our models are simplifications and we can ultimately judge whether the simplifications are warranted or not by the usefulness of the model. Still, we must understand that "usefulness" is not a single measurement and there will be trade-offs. Finally, the last quote cautions us that we must not, in the end, confuse our models with the process we modeled. Models do leave out details we hope are unimportant for our purposes but we may be wrong. Something that was unimportant when the model was built could later become important. We should constantly check our models to make sure they remain useful and that the system we're modeling hasn't undergone a change.

> Reports that say that something hasn't happened are always interesting to me, because as we know, there are known knowns; there are things we know we know. We also know there are known unknowns; that is to say we know there are some things we do not know. But there are also unknown unknowns – the ones we don't know we don't know. And if one looks throughout the history of our country and other free countries, it is the latter category that tend to be the difficult ones. -- Donald Rumsfeld

At the time, people made a bit of fun at Rumsfeld's expense but he's not wrong. There are the known knowns, the unknown knowns and the unknown unknowns. There is a fourth category that can be as dangerous as the second and third, the known untruths: thing we think we know to be true but are actually false.

So the four quotes describe a continuum that we should keep in mind when building models:

1. We cannot include everything in a model,
2. Models will therefore be incomplete and imperfect but still useful,
3. Because of a model is always simplification, it is not the process being modeled.
4. We continually need to check the model because the system may change.

Let's start with our EDA for synthetic data a few pages back:

In [3]:
np.random.seed(1476374623)

In [5]:
x1 = np.concatenate([stats.norm.rvs( 100.0, 10.0, 100), stats.norm.rvs(110.0, 20.0, 50)])

In [8]:
print("min       = {stat:.2f}".format(stat=np.min( x1)))
print("Q1        = {stat:.2f}".format(stat=stats.mstats.mquantiles( x1, [0.25])[0]))
print("Q2        = {stat:.2f}".format(stat=np.median( x1)),)
print("mean      = {stat:.2f}".format(stat=np.mean(x1)))
print("Q3        = {stat:.2f}".format(stat=stats.mstats.mquantiles( x1, [0.75])[0]))
print("max       = {stat:.2f}".format(stat=np.max( x1)))
print("range     = {stat:.2f}".format(stat=np.max( x1) - np.min(x1)))
print("IQR       = {stat:.2f}".format(stat=stats.mstats.mquantiles(x1, [0.75])[0] - 
                                stats.mstats.mquantiles(x1, [0.25])[0]))
print("var (std) = {var:.2f} ({std:.2f})".format(var=np.var( x1), std=np.std(x1)))
print("COV       = {stat:.2f}%".format(stat=(np.std(x1)/np.mean(x1))*100))

min       = 74.44
Q1        = 93.90
Q2        = 102.01
mean      = 103.99
Q3        = 112.49
max       = 161.92
range     = 87.48
IQR       = 18.60
var (std) = 226.25 (15.04)
COV       = 14.46%


It doesn't really matter what $x_1$ *really* is for now.
For our purposes, $x_1$ is just some measurement or event from a process that we're interested in learning about.
We have collected 150 such observations ($n=150$).

Looking at our descriptive statistics, we have a minimum of 74.44--this may or may not surprise us depending on our domain knowledge. If it's negative, that may indicate a problem with the data (or not). The maximum is 161.92.

25% of the data is below 93.9 (between 74.44 and 93.9). 50% is below 102.01 but 50% is above. The mean is 103.99. This is above the median so this data is "slightly" right skewed. The 3rd quartile is 112.49.

25% of the data is above 112.49 but not higher than the maximum, 161.92.

The data shows some dispersion. The coefficient of variation is 14.46%. This may or may not be typical of this problem or this data. Again, we may be able to make observations about the range, interquartile range, variance, and standard deviation based on our experience, the variable, domain knowledge, and or the problem.

The main point here is that although we have *described* the data, what happens if we want to know, what is the next value that this process will produce?

This or something like it is the real question we want to answer.
We're often interested in averages (means and rates) because we want to know what's going to happen next either in terms of individual observations or in terms of averages. If my conversion
rate is 2.3% this month, what will it be next month? A rate is just an average over binary data encoded as $\{0, 1\}$.

And so we start with the question, why is the mean a reasonable (initial?) model for this problem?

## The Mean as a Model

We start by asking, what is the mean of a continuous random variable, $x$? By this, we aren't asking, "what is the formula?" because we know what the formula is:

$\bar{x} = \frac{1}{n}\sum_i x_i$

 We're asking something a bit more fundamental, why are we interested in the mean at all?

If we look at this from a prediction point of view, and we say that we want to predict $x$,  what is a good value of $x$ to predict? One way to frame the question is to ask, what is the cost of getting the prediction wrong? This idea of cost is generalized by the concept of a *loss function*. Our best prediction of $x$, $\breve{x}$, is thus the one that minimizes our loss function, $L(\breve{x})$. A very popular loss function is *total squared loss*:

$L(\breve{x}) = \sum_i (\breve{x} - x_i)^2$

although, as we will see, it is not the only possible loss function.

If we want to find the $\breve{x}$ that minimizes this loss function, we need can apply a bit of the calculus by taking the partial derivative with respect to $\breve{x}$ and setting it to zero (0). (Without loss of generality, we can minimize (1/2) times the loss function...this will make the math work out nicer later. This is one of those math "tricks").

First, we take the partial derivative:

$\frac{\partial}{\partial\breve{x}}L(\breve{x}) = \frac{\partial}{\partial\breve{x}}\frac{1}{2}\sum_i (\breve{x} - x_i)^2$

$\frac{\partial}{\partial\breve{x}}L(\breve{x}) = 2 \frac{1}{2} \times \sum_i (\breve{x} - x_i)$

$\frac{\partial}{\partial\breve{x}}L(\breve{x}) = \sum_i \breve{x} - \sum_i x_i$

$\frac{\partial}{\partial\breve{x}}L(\breve{x}) = n\breve{x} - \sum_i x_i$

Next, we set the partial derivative to 0 and solve for $\breve{x}$:

$n\breve{x} - \sum_i x_i = 0$

$n\breve{x}  = \sum_i x_i$

$\breve{x}  = \frac{1}{n}\sum_i x_i$

This should look familiar. It's the formula for the mean. 

The implication is that the mean is the predicted value of $x$ that minimizes total squared loss. The formula for loss (squared error) should also look familiar:

$L(\breve{x}) = \sum_i (\breve{x} - x_i)^2$

Perhaps it will look more familiar if we take the *average* of it (which also does not affect the minimization):

$\frac{1}{n}L(\breve{x}) = \frac{1}{n} \sum_i (\breve{x} - x_i)^2$

average total squared loss is *variance* when $\breve{x}=\bar{x}$. 

In other words, we could not pick a better predictor of $x$ than $\bar{x}$ to minimize (average) squared loss; this is also the value that minimizes the measure of variance.

This might seem a bit strange to us because we generally think that we calculate the mean then the variance.
We don't think of the mean as something we pick to minimize variance.
Additionally, we just think of the variance as something we calculate and not actively minimize.

Note that other loss functions are possible. You don't always need to predict based minimizing (average) squared loss. As a result, you will end up with different models than the mean. For example,

1. If you want to minimize absolute error, you will pick the median. As the middle value, the median minimizes total absolute error.
2. If you want to minimize "0/1 Loss" (pronounced "zero one loss"), you will pick the mode. As the most common value, the mode minimizes "all or nothing" loss.